In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

# SET WORK DERC

# For my Mac
# os.chdir('//Users//patricklee//Desktop//파이썬//DACON//AI_workout')

# For my Desktop
os.chdir('C://Users//BIS_COM//data//dacon/workout/')

In [2]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')
submission=pd.read_csv('./sample_submission.csv')

In [3]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [4]:
import math

In [5]:
new_df = train.copy()

In [6]:
radian = 180 / math.pi

def cal_pitch (x,y,z):
    pitch = math.atan(x/math.sqrt((y**2)+(z**2)))*radian
    return pitch

def cal_roll (x,y,z):
    roll = math.atan(y/math.sqrt((x**2)+(z**2)))*radian
    return roll

def cal_angle_z (x,y,z):
    angle_z = math.atan(math.sqrt((x**2)+(y**2))/z)*radian
    return angle_z

def cal_gg_x (g_x):
    gg_x = g_x * 0.02
    return gg_x

def cal_gg_y (g_y):
    gg_y = g_y * 0.02
    return gg_y

def cal_gg_z (g_z):
    gg_z = g_z * 0.02
    return gg_z

In [7]:
new_df['pitch'] = new_df.apply(lambda x : cal_pitch(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
new_df['roll'] = new_df.apply(lambda x : cal_roll(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
new_df['angle_z'] = new_df.apply(lambda x : cal_angle_z(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
# gy
new_df['gg_x'] = new_df.apply(lambda x : cal_gg_x(x['gy_x']), axis=1)
new_df['gg_y'] = new_df.apply(lambda x : cal_gg_y(x['gy_y']), axis=1)
new_df['gg_z'] = new_df.apply(lambda x : cal_gg_z(x['gy_z']), axis=1)

<ipython-input-6-f010c9ec0a00>:12: RuntimeWarning: divide by zero encountered in double_scalars
  angle_z = math.atan(math.sqrt((x**2)+(y**2))/z)*radian


In [8]:
new_df

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,pitch,roll,angle_z,gg_x,gg_y,gg_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,79.073624,-8.396683,-83.058839,-0.011832,-0.610980,-0.633522
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,78.159835,-8.698302,-82.029235,0.006062,-0.782782,-0.498544
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,76.226605,-8.352177,-79.126797,-0.072346,-0.882451,-0.500393
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,76.298763,-9.967037,-80.694753,0.054260,-1.071957,-0.549080
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,77.384519,-8.096783,-80.390898,0.085734,-1.158131,-0.559225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,-44.665405,-40.505406,73.156120,-0.587357,-2.080273,-1.525809
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,-42.855425,-40.971520,70.862773,-0.602982,-2.035936,-1.532502
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,-41.189947,-41.333028,68.854788,-0.557462,-1.975521,-1.587302
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,-39.017186,-42.961261,68.092120,-0.472731,-1.982790,-1.605190


In [9]:
new_df.describe()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,pitch,roll,angle_z,gg_x,gg_y,gg_z
count,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06,1.875000e+06
mean,1.562000e+03,2.995000e+02,-4.489683e-01,-1.617913e-01,5.571471e-02,-1.307982e+00,-4.337745e-01,8.864100e-01,-3.023021e+01,-1.031714e+01,1.326217e+00,-2.615964e-02,-8.675489e-03,1.772820e-02
std,9.021100e+02,1.732049e+02,7.173082e-01,5.872018e-01,4.724824e-01,6.083228e+01,7.329832e+01,7.063345e+01,4.461678e+01,3.387248e+01,7.107417e+01,1.216646e+00,1.465966e+00,1.412669e+00
min,0.000000e+00,0.000000e+00,-8.838427e+00,-9.299714e+00,-8.316425e+00,-1.811127e+03,-1.594256e+03,-1.346143e+03,-8.999223e+01,-8.998953e+01,-9.000000e+01,-3.622253e+01,-3.188511e+01,-2.692286e+01
25%,7.810000e+02,1.497500e+02,-9.625643e-01,-5.163020e-01,-2.123550e-01,-2.048984e+01,-1.865320e+01,-1.546226e+01,-6.917862e+01,-3.165264e+01,-7.607535e+01,-4.097968e-01,-3.730640e-01,-3.092451e-01
50%,1.562000e+03,2.995000e+02,-6.560482e-01,-2.033985e-01,3.031258e-02,-1.421819e+00,-2.642753e-01,-1.798643e-01,-4.160925e+01,-1.158522e+01,2.460755e+01,-2.843638e-02,-5.285506e-03,-3.597285e-03
75%,2.343000e+03,4.492500e+02,-1.701162e-04,1.406493e-01,3.581982e-01,1.572245e+01,1.676328e+01,1.641878e+01,-1.008139e-02,7.972242e+00,7.293201e+01,3.144491e-01,3.352656e-01,3.283756e-01
max,3.124000e+03,5.990000e+02,8.426799e+00,8.484622e+00,6.432668e+00,1.870256e+03,2.111657e+03,1.255849e+03,8.980285e+01,8.999319e+01,9.000000e+01,3.740512e+01,4.223314e+01,2.511698e+01


In [7]:
new_df.to_csv('new_value_df.csv',index=False)

In [8]:
new_test_df = test.copy()

In [9]:
new_test_df['pitch'] = new_test_df.apply(lambda x : cal_pitch(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
new_test_df['roll'] = new_test_df.apply(lambda x : cal_roll(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
new_test_df['angle_z'] = new_test_df.apply(lambda x : cal_angle_z(x['acc_x'],x['acc_y'],x['acc_z']), axis=1)
# gy
new_test_df['gg_x'] = new_test_df.apply(lambda x : cal_gg_x(x['gy_x']), axis=1)
new_test_df['gg_y'] = new_test_df.apply(lambda x : cal_gg_y(x['gy_y']), axis=1)
new_test_df['gg_z'] = new_test_df.apply(lambda x : cal_gg_z(x['gy_z']), axis=1)

<ipython-input-6-f010c9ec0a00>:12: RuntimeWarning: divide by zero encountered in double_scalars
  angle_z = math.atan(math.sqrt((x**2)+(y**2))/z)*radian


In [10]:
new_test_df.to_csv('new_test_df.csv',index=False)